In [1]:
import numpy as np
import pandas as pd
import wfdb
import ast
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
import os

from sklearn.utils import shuffle
import math
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,normalize, MinMaxScaler


import os
import wandb
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score
import warnings

In [1]:
### Preprocessing 
#   Using the super classes, multi label classification, excluding samples with no labels and considering atleast one label

path = 'ptb/'
Y = pd.read_csv(path+ 'ptbxl_database.csv', index_col = 'ecg_id')



data = np.array([wfdb.rdsamp(path+f)[0] for f in Y.filename_lr])
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
    
agg_df = pd.read_csv(path+ 'scp_statements.csv', index_col = 0)

agg_df = agg_df[agg_df.diagnostic == 1]

def agg(y_dic):
    temp =[]
    
    for key in y_dic.keys():
        if y_dic[key] in [100, 80, 0] :
            if key in agg_df.index:
                if key in ['ASMI', 'IMI']:

                    temp.append(key)
    return list(set(temp))


Y['diagnostic_subclass'] = Y.scp_codes.apply(agg)
Y['subdiagnostic_len'] = Y['diagnostic_subclass'].apply(lambda x: len(x))

## MI
x_1 = data[Y['subdiagnostic_len'] == 1]
y_1 = Y[Y['subdiagnostic_len'] == 1]

def norm_agg(y_dic):
    temp =[]
    
    for key in y_dic.keys():
        if y_dic[key] in [100] :
            if key == 'NORM':
                return 'NORM'
    
Q = Y.copy()
Q['diagnostic_subclass'] = Y.scp_codes.apply(norm_agg)


## Norm
x_2 = data[Q['diagnostic_subclass'] == 'NORM']
y_2 = Q[Q['diagnostic_subclass'] == 'NORM'] 


x_1_train = x_1[y_1.strat_fold <= 8]
y_1_train = y_1[y_1.strat_fold <= 8]

x_1_test = x_1[y_1.strat_fold > 8]
y_1_test = y_1[y_1.strat_fold > 8]

x_2_train = x_2[y_2.strat_fold <= 2][:800]
y_2_train = y_2[y_2.strat_fold <= 2][:800]

x_2_test = x_2[y_2.strat_fold == 3][:200]
y_2_test = y_2[y_2.strat_fold == 3][:200]

train_data = np.concatenate((x_1_train, x_2_train), axis = 0)
test_data = np.concatenate((x_1_test, x_2_test), axis = 0)

y_1_train.diagnostic_subclass = y_1_train.diagnostic_subclass.apply(lambda x : x[0])
y_1_test.diagnostic_subclass = y_1_test.diagnostic_subclass.apply(lambda x : x[0])

train_label = np.concatenate((y_1_train.diagnostic_subclass.values, y_2_train.diagnostic_subclass.values), axis = 0)
test_label = np.concatenate((y_1_test.diagnostic_subclass.values, y_2_test.diagnostic_subclass.values), axis = 0)


le = LabelEncoder()
train_label = to_categorical(le.fit_transform(train_label))
test_label = to_categorical(le.transform(test_label))

train_data, train_label = shuffle(train_data, train_label, random_state = 42)

In [4]:

# Standardizing

def apply_scaler(X, scaler):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(scaler.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp


scaler = StandardScaler()

scaler.fit(np.vstack(train_data).flatten()[:,np.newaxis].astype(float))

X_train_scale = apply_scaler(train_data, scaler)
X_test_scale = apply_scaler(test_data, scaler)

del train_data, test_data, data

In [5]:
class DataGen(tf.keras.utils.Sequence):
    def __init__(self, X, y,batch_size = 16):
        self.batch_size = batch_size
        self.X = X
        self.y = y
        
    def __len__(self):
        return math.ceil(len(self.X) / self.batch_size)
    
    def __getitem__(self,idx):
        
        X_full = self.X[idx * self.batch_size:(idx + 1) *self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *self.batch_size]
        

        return np.transpose(X_full[..., np.newaxis], (0, 2, 1, 3)) ,batch_y
    
## Params

batch_size = 32
    
train_gen = DataGen(X_train_scale, train_label, batch_size = batch_size)
test_gen = DataGen(X_test_scale, test_label, batch_size = batch_size)

In [6]:
test = train_gen[0][0].shape
print(test)

(32, 12, 1000, 1)


In [7]:
import tensorflow.keras.backend as K

class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences = False, dim = 32, **kwargs):
        self.return_sequences = return_sequences
        self.dim = dim
        super(attention,self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1], self.dim),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1], self.dim),
                               initializer="zeros")
        self.V = self.add_weight(name = "Vatt", shape = (self.dim, 1), initializer = "normal")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        e = K.dot(e, self.V)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences :
            return output, a
        
        return K.sum(output, axis=1), a

    def get_config(self):
        base_config = super().get_config()
        config = {"return sequences" : tf.keras.initializers.serialize(self.return_sequences), "att dim" : tf.keras.initializers.serialize(self.dim)}
        return dict(list(base_config.items()) + list(config.items()))
    
## Resnet blocks

def relu_bn(inputs: tf.Tensor) -> tf.Tensor:
    
    
    dp = Dropout(0.5)(inputs)
    relu = ReLU()(dp)
    bn = BatchNormalization()(relu)
    return bn


def residual_block(x: tf.Tensor, downsample: bool, filters: int, kernel_size: int = 12) -> tf.Tensor:
    
    y = Conv1D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv1D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv1D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

In [8]:
## Params

sig_len = 1000
beat_size = 50

In [9]:
from tensorflow.keras.layers import Conv1D, Input, Attention, LSTM, Activation, Dense, Average,ReLU, BatchNormalization,Add, Reshape, Bidirectional, Concatenate

num_channel = 12
num_filters = 32
num_blocks_list = [2, 2, 2]

inputs = Input(shape = (num_channel, sig_len, 1), batch_size = None)

#### Beat Level 
x = K.reshape(inputs, (-1, beat_size,1 ))

x = Conv1D(32 ,12 ,padding = 'same')(x)
x = Activation('relu')(x)

for i in range(len(num_blocks_list)):
    num_blocks = num_blocks_list[i]
    for j in range(num_blocks):
        x = residual_block(x, downsample=(j==0 and i!=0), filters=num_filters)
    num_filters *= 2
    
x, _ = attention(name = "beat_att")(x)

##### Rhythm level
x = K.reshape(x,(-1, int(sig_len/beat_size) , 64))

x = Bidirectional(LSTM(32, return_sequences = True))(x)
x, _ = attention(name = "rhythm_att")(x)


#### Channel level

x = K.reshape(x, (-1, num_channel, 64))
x, _ = attention(name = "channel_att")(x)

outputs = Dense(5, activation = 'sigmoid')(x)

aux_model = tf.keras.models.Model(inputs = inputs, outputs = outputs)

aux_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy',tf.keras.metrics.AUC(multi_label = True)])
aux_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 1000, 1) 0                                            
__________________________________________________________________________________________________
tf_op_layer_Reshape (TensorFlow [(None, 50, 1)]      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 50, 16)       144         tf_op_layer_Reshape[0][0]        
__________________________________________________________________________________________________
activation (Activation)         (None, 50, 16)       0           conv1d[0][0]                     
_______________________________________________________________________________________

In [10]:
if not os.path.exists('MI_subtypes'):
    os.mkdir('MI_subtypes')    

In [2]:
# wandb.init(project = '3_level ASMI, IMI and NORM', name = 'original_2')

In [10]:
outputs = Dense(3, activation='softmax')(aux_model.layers[-2].output[0])

model = tf.keras.models.Model(inputs = aux_model.input, outputs = outputs)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy',tf.keras.metrics.AUC()])

model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 12, 1000, 1) 0                                            
__________________________________________________________________________________________________
tf_op_layer_Reshape (TensorFlow [(None, 50, 1)]      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 50, 16)       144         tf_op_layer_Reshape[0][0]        
__________________________________________________________________________________________________
activation (Activation)         (None, 50, 16)       0           conv1d[0][0]                     
_______________________________________________________________________________________

In [13]:
### Accuracy metric

def metrics(y_true, y_scores):
    y_pred = y_scores >= 0.5
    acc = np.zeros(y_pred.shape[-1])
    
    for i in range(y_pred.shape[-1]):
        acc[i] = accuracy_score(y_true[:,i], y_pred[:,i])
    return acc, np.mean(acc)

## Callback for logging and metrics 

class model_checkpoint(tf.keras.callbacks.Callback):

    def __init__(self, filepath, gen, monitor='loss',  options=None, **kwargs):

        super().__init__()

        self.filepath = filepath
        self.monitor = monitor
        self.test_data = gen
        
        
    def on_epoch_end(self, epoch, logs = {}) :
        
        test_len = len(self.test_data)
        score = []
        gt =[]

        for i in range(test_len):
            X,y = self.test_data[i][0], self.test_data[i][1]
            temp_score = self.model.predict(X)
            score.append(temp_score)
            gt.append(y)

        score = np.concatenate(score, axis = 0)
        gt = np.concatenate(gt, axis = 0)
        
        roc_auc = roc_auc_score(gt, score)
        _, accuracy = metrics(gt, score)
        
        temp_path = f"{epoch+1}_roc_{roc_auc:.4f}.h5"
        path = os.path.join(self.filepath, temp_path)
        
        
        self.model.save_weights(path)

        wandb.log({'train_loss' : logs['loss'], 'epoch' : epoch})
        wandb.log({'train_keras_auroc' : logs.get(self.monitor), 'epoch' : epoch})
        
        wandb.log({'test_loss' : logs['val_loss'], 'epoch' : epoch})
        wandb.log({'test_keras_auroc' : logs['val_auc_1'], 'epoch' : epoch})

        wandb.log({'test_roc_score' : roc_auc, 'epoch' : epoch})
        wandb.log({'test_accuracy_score' : accuracy, 'epoch' : epoch})
        
        logs['val_roc_auc'] = roc_auc
        logs['val_accuracy_score'] = accuracy
    
    def set_model(self, model):
        self.model = model

        
metric = 'auc_1'
checkpoint_filepath = 'MI_subtypes'

checkpoint = model_checkpoint(checkpoint_filepath, monitor = metric, gen = test_gen )

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.1,
        patience=10,
        min_lr=0.001 * 0.001)

callbacks = [checkpoint, reduce_lr]     

In [ ]:
history = model.fit(train_gen, epochs = 60, callbacks = callbacks, validation_data = test_gen)

Epoch 1/60
86/86 [==============================] - 23s 266ms/step - loss: 0.6755 - accuracy: 0.7164 - auc_1: 0.8839 - val_loss: 1.7411 - val_accuracy: 0.2616 - val_auc_1: 0.4793 - val_roc_auc: 0.5341 - val_accuracy_score: 0.5077 - lr: 0.0010
Epoch 2/60
86/86 [==============================] - 19s 224ms/step - loss: 0.4946 - accuracy: 0.8002 - auc_1: 0.9345 - val_loss: 0.9482 - val_accuracy: 0.6061 - val_auc_1: 0.7606 - val_roc_auc: 0.7567 - val_accuracy_score: 0.7661 - lr: 0.0010
Epoch 3/60
86/86 [==============================] - 19s 226ms/step - loss: 0.4036 - accuracy: 0.8460 - auc_1: 0.9559 - val_loss: 1.2280 - val_accuracy: 0.6045 - val_auc_1: 0.7837 - val_roc_auc: 0.8271 - val_accuracy_score: 0.7443 - lr: 0.0010
Epoch 4/60
86/86 [==============================] - 19s 223ms/step - loss: 0.3664 - accuracy: 0.8617 - auc_1: 0.9629 - val_loss: 0.5764 - val_accuracy: 0.7719 - val_auc_1: 0.9137 - val_roc_auc: 0.9061 - val_accuracy_score: 0.8458 - lr: 0.0010
Epoch 5/60
86/86 [==========

In [11]:
path_weights = r'C:\Users\likit\OneDrive\Desktop\Cardio-Viz\Code\main\multi_level_3_level\AMI, IMI and NORM -- new\original\59_roc_0.9586.h5'

model.load_weights(path_weights)

In [12]:
y_test = test_label
test_gen = DataGen(X_test_scale, y_test, batch_size = len(y_test))

pred = model.predict(test_gen[0][0])

roc_auc_score(y_test, pred)

0.9585699858262955

In [13]:
y_pred = np.argmax(pred, axis = 1)

confusion = confusion_matrix(np.argmax(y_test, axis = 1), y_pred)

# np.argmax(y_test, axis = 1), y_pred

confusion, np.bincount(np.argmax(y_test, axis = 1)), le.classes_

(array([[236,  18,   9],
        [ 30, 115,  19],
        [  5,   8, 187]], dtype=int64),
 array([263, 164, 200], dtype=int64),
 array(['ASMI', 'IMI', 'NORM'], dtype=object))

In [14]:
### Accuracy metric

def metrics(y_true, y_scores):
    y_pred = y_scores >= 0.5
    acc = np.zeros(y_pred.shape[-1])
    
    for i in range(y_pred.shape[-1]):
        acc[i] = accuracy_score(y_true[:,i], y_pred[:,i])
    return acc, np.mean(acc)

acc, mean_acc = metrics(y_test, pred)
print(f'class wise accuracy: {acc}')
print(f'accuracy: {mean_acc}')

class wise accuracy: [0.90590112 0.87878788 0.93460925]
accuracy: 0.9064327485380117


In [15]:
### Class wise AUC

roc_score = roc_auc_score(y_test, pred, average='macro')
print(f'roc_score : {roc_score}')

def AUC(y_true: np.ndarray, y_pred: np.ndarray, verbose=False) -> float:
    """Computes the macro-average AUC score.

    Args:
        y_true (np.ndarray): list of labels
        y_pred (np.ndarray): list of predicted probabilities

    Returns:
        float: macro-average AUC score.
    """
    aucs = []
    assert len(y_true.shape) == 2 and len(y_pred.shape) == 2, 'Predictions and labels must be 2D.'
    for col in range(y_true.shape[1]):
        try:
            aucs.append(roc_auc_score(y_true[:, col], y_pred[:, col]))
        except ValueError as e:
            if verbose:
                print(
                    f'Value error encountered for label {col}, likely due to using mixup or '
                    f'lack of full label presence. Setting AUC to accuracy. '
                    f'Original error was: {str(e)}.'
                )
            aucs.append((y_pred == y_true).sum() / len(y_pred))
    return np.array(aucs)

class_auc = AUC(y_test, pred)
print(f'class wise AUC : {class_auc}')



roc_score : 0.9585699858262955
class wise AUC : [0.96021184 0.92918664 0.98631148]


In [16]:
def multi_threshold_precision_recall(y_true: np.ndarray, y_pred: np.ndarray, thresholds: np.ndarray) :
    
    # Expand analysis to number of thresholds
    y_pred_bin = np.repeat(y_pred[None, :, :], len(thresholds), axis=0) >= thresholds[:, None, None]
    y_true_bin = np.repeat(y_true[None, :, :], len(thresholds), axis=0)

    # Compute true positives
    TP = np.sum(np.logical_and(y_true, y_pred_bin), axis=2)

    # Compute macro-average precision handling all warnings
    with np.errstate(divide='ignore', invalid='ignore'):
        den = np.sum(y_pred_bin, axis=2)
        precision = TP / den
        precision[den == 0] = np.nan
        with warnings.catch_warnings():  # for nan slices
            warnings.simplefilter("ignore", category=RuntimeWarning)
            av_precision = np.nanmean(precision, axis=1)

    # Compute macro-average recall
    recall = TP / np.sum(y_true_bin, axis=2)
    av_recall = np.mean(recall, axis=1)

    return av_precision, av_recall


def metric_summary(y_true: np.ndarray, y_pred: np.ndarray, num_thresholds: int = 10) :
    
    thresholds = np.arange(0.00, 1.01, 1. / (num_thresholds - 1), float)
    average_precisions, average_recalls = multi_threshold_precision_recall(
        y_true, y_pred, thresholds
    )
    f_scores = 2 * (average_precisions * average_recalls) / (average_precisions + average_recalls)
    auc = np.array(AUC(y_true, y_pred, verbose=True)).mean()
    return (
        f_scores[np.nanargmax(f_scores)],
        auc,
        f_scores,
        average_precisions,
        average_recalls,
        thresholds
    )

metric_summary(y_test, pred)


(0.8758032167085021,
 0.9585699858262955,
 array([0.5       , 0.87537245, 0.87580322, 0.87309328, 0.8611916 ,
        0.85737439, 0.84317032, 0.82782609, 0.78438662,        nan]),
 array([0.33333333, 0.81472621, 0.84077618, 0.85247209, 0.85955056,
        0.87149918, 0.89445438, 0.91013384, 0.93986637,        nan]),
 array([1.        , 0.94577352, 0.9138756 , 0.89473684, 0.86283892,
        0.84370016, 0.79744817, 0.75917065, 0.67304625, 0.        ]),
 array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
        0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]))

In [17]:
pred_values = pred >= 0.5

report = classification_report(y_test, pred_values, target_names = le.classes_)
print(report)

              precision    recall  f1-score   support

        ASMI       0.88      0.90      0.89       263
         IMI       0.81      0.70      0.75       164
        NORM       0.87      0.93      0.90       200

   micro avg       0.86      0.85      0.86       627
   macro avg       0.86      0.84      0.85       627
weighted avg       0.86      0.85      0.86       627
 samples avg       0.85      0.85      0.85       627

C:\Users\likit\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from scipy.signal import resample

# Plotting
path = 'ptb/'
sample = 'record_sample'

test_data = np.array([wfdb.rdsamp(path+sample)[0] ])
test_data_scale = apply_scaler(test_data, scaler)

test_data_scale = np.transpose(test_data_scale[..., np.newaxis], (0,2,1,3))
    

### To get layer names
# for layer in model.layers:
#     print(layer.name)

attention_layer = tf.keras.models.Model(inputs = model.input, outputs = [model.get_layer("beat_att").output, 
                                                                                        model.get_layer("rhythm_att").output,
                                                                                        model.get_layer("channel_att").output])
beat, rhythm, channel = attention_layer(test_data_scale)

beat_att = np.asarray(beat[1]); rhythm_att = np.asarray(rhythm[1]); channel_att = np.asarray(channel[1])

beat_att = beat_att.reshape(240, 13)
beat_only_att = np.empty((240,beat_size))
for i in range(beat_att.shape[0]):
    beat_only_att[i] = resample(beat_att[i], beat_size)

beat_att = np.copy(beat_only_att)

## Rhytm
rhythm_att = rhythm_att.reshape(12*20)
for i in range(12*20):
    beat_att[i] = beat_att[i] * rhythm_att[i]


# Channel
beat_att = beat_att.reshape(12, 20*50)
channel_att = channel_att.flatten()
for i in range(12):
    beat_att[i] = beat_att[i] * channel_att[i]
    
scores = np.copy(beat_att)

In [3]:
model(test_data_scale), le.classes_

In [14]:
### Calculate Beat level using channel level

beat_channel = np.copy(beat_only_att.reshape(12, 20*50))

for i in range(12):
    beat_channel[i] = beat_channel[i] * channel_att[i]
    

In [5]:
scores_nor = (scores.flatten() - scores.flatten().min(keepdims=True)) / (scores.flatten().max( keepdims=True) - scores.flatten().min(keepdims=True))
scores_nor = scores_nor.reshape(12, 1000)


beat_only_att_nor = (beat_only_att.flatten() - beat_only_att.flatten().min(keepdims=True)) / (beat_only_att.flatten().max( keepdims=True) - beat_only_att.flatten().min(keepdims=True))
beat_only_att_nor = beat_only_att_nor.reshape(12, 1000)
beat_only_att = beat_only_att.reshape(12, 1000)

ch_info = ['I',
           'II',
           'III',
           'AVR',
           'AVL',
           'AVF',
           'V1',
           'V2',
           'V3',
           'V4',
           'V5',
           'V6']

import matplotlib.pyplot as plt

fig, axs = plt.subplots(12, figsize = (45, 30))
x = np.arange(1000)


fig, axs = plt.subplots(12, figsize = (35, 25))
x = np.arange(1000)

for i, (ax, ch) in enumerate(zip(axs, ch_info)):
    im = ax.scatter(np.arange(len(test_data[:,:,i].squeeze())), test_data[:,:,i].squeeze(), cmap = 'hot_r', c= beat_channel[i])
    plt.colorbar(im, ax = ax)
    ax.plot(test_data[:,:,i].squeeze())
    ax.set_title(ch, fontsize = 30)


In [4]:
import plotly.express as px
import plotly.offline as pyo

pyo.init_notebook_mode()

fig = px.bar(channel_att)
fig.show()